In [15]:
# %%
# import torch
# from numba import cuda
# cuda.select_device(0)
# cuda.close()
# cuda.select_device(0)
# import os
# os.system("bash /home/peterr/clean_tmp_script.sh")
# torch.cuda.empty_cache()


In [16]:

# %%
import pandas as pd
import numpy as np
import os
f = "transfer_10.csv"
data_dir = "/home/peterr/macocu/task8/transfer_10/"
df = pd.read_csv(f)
modelname = "300_"
df["path"] = data_dir + df["hashname"]
df["sentence"] = df.human_transcript
df = df.loc[:1000, ["path", "sentence"]]

# Use old or new vocab?
os.system("cp vocab_old.json vocab.json")


0

In [9]:



# %%
from sklearn.model_selection import train_test_split

common_voice_train_df, common_voice_test_df = train_test_split(df, test_size=500, random_state=42)


common_voice_train_df.reset_index(drop=True, inplace=True)
common_voice_test_df.reset_index(drop=True, inplace=True)

import datasets
from datasets import load_dataset, load_metric, Audio
def load_audio(path):
    return datasets.Audio(sampling_rate=16000).decode_example(path)

# Adding audio
common_voice_train_df.loc[:, "audio"] = common_voice_train_df.path.apply(load_audio)
common_voice_test_df.loc[:, "audio"] = common_voice_test_df.path.apply(load_audio)


In [12]:

# Initiating a dataset from pandas dataframe
common_voice_train_dataset = datasets.Dataset.from_pandas(common_voice_train_df)
common_voice_test_dataset = datasets.Dataset.from_pandas(common_voice_test_df)


In [14]:
common_voice_train_dataset[0]

{'path': '/home/peterr/macocu/task8/transfer_10/5T4HXO1WsDU_2830.21-2850.13.wav',
 'sentence': 'treba stvoriti pretpostavke da imao novac koji ovdje dajemo onima koji inače primaju najveće mirovine i najveće plaće bez potrebe i bez da nas je to itko pitao i suprotno našim predizbornim obećanjima svih stranaka ovdje se ovo radi dakle potpuno odstupamo',
 'audio': {'array': [-0.017242431640625,
   -0.01251220703125,
   0.00054931640625,
   0.01605224609375,
   0.02593994140625,
   0.026397705078125,
   0.02056884765625,
   0.01837158203125,
   0.020477294921875,
   0.016754150390625,
   0.01123046875,
   0.0126953125,
   0.01422119140625,
   0.01513671875,
   0.018463134765625,
   0.02191162109375,
   0.02496337890625,
   0.02276611328125,
   0.016143798828125,
   0.0028076171875,
   -0.0074462890625,
   -0.010955810546875,
   -0.00982666015625,
   -0.007476806640625,
   -0.005157470703125,
   -9.1552734375e-05,
   0.0013427734375,
   -0.004150390625,
   -0.007598876953125,
   -0.0076904

In [5]:

# %%
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer)

import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


from transformers import Trainer
from transformers import TrainingArguments
from transformers import Wav2Vec2ForCTC
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass, field
import torch


def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(
        audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch


common_voice_train_mapped = common_voice_train_dataset.map(
    prepare_dataset, remove_columns=common_voice_train_dataset.column_names)
common_voice_test_mapped = common_voice_test_dataset.map(
    prepare_dataset, remove_columns=common_voice_test_dataset.column_names)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/24501 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

In [7]:

# %%
repo_name = modelname
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

max_input_length_in_sec = 20
common_voice_train_mapped = common_voice_train_mapped.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m",
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

model.freeze_feature_extractor()

from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=8,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=800,
  logging_steps=400,
  learning_rate=6e-4,
  warmup_steps=500,
  save_total_limit=1,
  push_to_hub=False,
)

from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train_mapped,
    eval_dataset=common_voice_test_mapped,
    tokenizer=processor.feature_extractor,
)

trainer.train()




  0%|          | 0/25 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/facebook/wav2vec2-xls-r-300m/resolve/main/config.json from cache at /home/peterr/.cache/huggingface/transformers/dabc27df63e37bd2a7a221c7774e35f36a280fbdf917cf54cadfc7df8c786f6f.a3e4c3c967d9985881e0ae550a5f6f668f897db5ab2e0802f9b97973b15970e6
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,

Step,Training Loss,Validation Loss,Wer
800,0.261200,0.162286,0.940000
1600,0.143700,0.140359,0.914000
2400,0.088100,0.141079,0.882000


Saving model checkpoint to 300_/checkpoint-400
Configuration saved in 300_/checkpoint-400/config.json
Model weights saved in 300_/checkpoint-400/pytorch_model.bin
Feature extractor saved in 300_/checkpoint-400/preprocessor_config.json
The following columns in the evaluation set  don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to 300_/checkpoint-800
Configuration saved in 300_/checkpoint-800/config.json
Model weights saved in 300_/checkpoint-800/pytorch_model.bin
Feature extractor saved in 300_/checkpoint-800/preprocessor_config.json
Deleting older checkpoint [300_/checkpoint-400] due to args.save_total_limit
Saving model checkpoint to 300_/checkpoint-1200
Configuration saved in 300_/checkpoint-1200/config.json
Model weights saved in 300_/checkpo

TrainOutput(global_step=3032, training_loss=0.4600355276017202, metrics={'train_runtime': 49525.3215, 'train_samples_per_second': 3.916, 'train_steps_per_second': 0.061, 'total_flos': 9.513123799290988e+19, 'train_loss': 0.4600355276017202, 'epoch': 8.0})